<a href="https://colab.research.google.com/github/abdulkadirdemirci/AB_Testning/blob/main/AB_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####################################
# IS PROBLEMI

Facebook kisa sure once mevcut "maximumbidding"
adi verilen teklif verme turune alternatif olarak
yeni bir teklif turu olan "averagebidding"’i tanitti.
Musterilerimizden biri olanbombabomba.com, 
bu yeni ozelligi test etmeye karar verdi ve
averagebidding'in maximumbidding'den daha fazla donusum 
getirip getirmediginianlamak icin birA/B testiyapmak istiyor.
A/B testi 1 aydir devam ediyor ve bombabomba.com
simdi sizden bu A/B testinin sonuclarini analiz etmenizi bekliyor.
Bombabomba.com icin nihai basari olcutu Purchase'dir. 
Bu nedenle, istatistiksel testler icin Purchase metrigine odaklanilmalidir.
#####################################


#####################################
# VERİ SETİ HİKAYESİ

Bir firmanin web site bilgilerini iceren bu veri setinde kullanicilarin
gordukleri ve tikladiklari reklam sayilari gibi bilgilerin yani sira 
buradan gelen kazanc bilgileri yer almaktadir.Kontrol ve Test grubu olmak
uzere iki ayri veri seti vardir. Bu veri setleriab_testing.xlsx excel’inin
ayri sayfalarinda yer almaktadir. Kontrol grubuna Maximum Bidding, 
test grubuna AverageBidding uygulanmistir.

* IMPRESSION : **REKLAM GORUNTULEM SAYISI**
* CLICK      : **GORUNTULENEN REKLAMA TIKLAMA SAYISI**
* PURCHASE   : **TIKLANILAN REKLAMLAR SONRASI SATIN ALININ URUN SAYISI**
* EARNING    : **SATIN ALINAN URUNLER SONRASI ELDE EDİLEN KAZANC**


# GEREKLI KUTUPHANELER 

In [1]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, \
    mannwhitneyu, pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

#####################################
# GOREV 1: VERIYI ANALIZ ETME
#####################################
# GOREV 1 ADIM 1:
* TODO: veri setini oku
* TODO: control ve test veri setini ayrı degiskenlere ata

#####################################

In [3]:
df_control_ = pd.read_excel("/content/drive/MyDrive/veriler/ab_testing.xlsx",sheet_name="Control Group")
df_test_    = pd.read_excel("/content/drive/MyDrive/veriler/ab_testing.xlsx",sheet_name="Test Group")
df_control = df_control_.copy()
df_test = df_test_.copy()

#####################################
# GOREV 1 ADIM 2:
* TODO: control ve test grubu verilerini analiz ediniz

#####################################

## ---- control set ----

In [5]:
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [6]:
df_control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [7]:
df_control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [8]:
df_control.shape

(40, 4)

## ---- test set ----

In [9]:
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [10]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [12]:
df_test.shape

(40, 4)

#####################################
# GOREV 1 ADIM 3:
* TODO: analizden sonra concat metodunu kullanarak ve control verilerini birlestiriniz

#####################################

## ---- pd.series'e cevirme islemi

In [13]:
df_concat = pd.concat([df_control,df_test],keys=["control","test"])

In [14]:
df_concat.reset_index(inplace=True)

In [15]:
df_concat.drop("level_1",axis=1,inplace=True)

In [16]:
df_concat.columns = ["dtype","impression","click","purchase","earning"]

In [17]:
df_concat.sample(10)

,dtype,impression,click,purchase,earning
18,control,82067.899936,6609.183433,267.028943,1791.798984
37,control,123678.934234,3649.073787,476.168128,2187.721217
32,control,101530.950229,5626.901227,555.002024,2090.371280
17,control,147539.336329,3856.726853,329.377032,2143.589914
15,control,101616.465549,5323.548253,801.795020,2182.161708
65,test,113732.668458,3251.849039,610.742322,2366.822411
19,control,81872.985723,7959.125069,729.161549,1331.696457
27,control,136171.271652,5784.321927,772.032879,1704.134541
29,control,94923.683705,5678.981962,468.878031,1857.201862
3,control,109914.400398,4910.882240,487.090773,1696.229178


#####################################
# GOREV 2: A/B TESTININ HIPOTEZLERININ TANIMLANMASI
#####################################
# GOREV 2 ADIM 1:
* TODO: hipotezleri tanımla

#####################################


**oran olusturulur:** 

**click/impression:** reklama tiklayanlar / goruntuleyenler


H0: control ve test veri seti arasında click/impression oranlari bakimindan istatistiksel olrak anlamlı bir farklılık yoktur.

H1: control ve test veri seti arasında click/impression oranlari bakimindan istatistiksel olrak anlamlı bir farklılık vardir.

 H0: p1 = p2
 
 H1: p1 != p2

#####################################
# GOREV 2 ADIM 2:
* TODO: kontrol ve test grubu icin purchase ortalamalarini analiz edin

#####################################

In [18]:
p = df_concat.groupby("dtype").agg({"click":"sum",
                                "impression":"sum",
                                "purchase":"mean"})

In [19]:
p

,click,impression,purchase
dtype,,,
control,204026.294903,4.068458e+06,550.894059
test,158701.990432,4.820496e+06,582.106097


In [20]:
p1_control = 204026.2949/4068457.9627
p2_test    = 158701.9904/4820496.4703

In [23]:
reklama_tiklayan = p.values[:,0]
reklami_goruntuleyen = p.values[:,1]

In [24]:
reklama_tiklayan

array([204026.29490309, 158701.99043224])

In [25]:
reklami_goruntuleyen

array([4068457.96270789, 4820496.47030138])

In [21]:
p1_control

0.05014831092530192

In [22]:
p2_test

0.03292233307871778

In [26]:
df_concat["ci_ratio"] = df_concat["click"] / df_concat["impression"]


In [27]:
df_concat.groupby("dtype").agg({"ci_ratio":"mean",
                                "purchase":"mean"})

,ci_ratio,purchase
dtype,,
control,0.053618,550.894059
test,0.034176,582.106097


#####################################
# GOREV 3: HIPOTEZ TESTININ GERCEKLESTIRILMESI
#####################################
# GOREV 3 ADIM 1:
* TODO: test ve kontrol gruplari icin varsayim kontrolu yap

#####################################

## ---- normallik varsayimi ----

 H0: standart normal dagilim ile control/test veri setinin purchase degiskeninin dagilimi arasında istatistiksel olarak anlamli bir farklilik yoktur.
 
 H1: standart normal dagilim ile control/test veri setinin purchase degiskeninin dagilimi arasında istatistiksel olarak anlamli bir farklilik vardir.

 H0: dagilim normal dagilima uyar.
 
 H1: dagilim normal dagilima uymaz.

In [28]:
for type in list(df_concat["dtype"].unique()):
    test_stat, p_value = shapiro(df_concat.loc[df_concat["dtype"]==type,"purchase"])
    print(type,"veri seti :","test degeri : %.4f,  p value : %.4f" % (test_stat,p_value))

control veri seti : test degeri : 0.9773,  p value : 0.5891
test veri seti : test degeri : 0.9589,  p value : 0.1541


## ---- normallik varsayimi sonuc ----


p value > 0.05 H0 reddedilemez. (normallik varsayimi saglanmistir.)

## ---- varyans homojenligi varsayimi ----

H0: varyans homojenligi saglanmistir.

H1: varyans homojenligi saglanmamistir.

In [29]:
test_stat, p_value = levene(df_concat.loc[df_concat["dtype"]=="control","purchase"],
                            df_concat.loc[df_concat["dtype"]=="test","purchase"])
print("test degeri : %.4f,  p value : %.4f" % (test_stat,p_value))

test degeri : 2.6393,  p value : 0.1083


## ---- varyans homojenligi varsayimi sonuc ----

p value > 0.05 H0 reddedilemez. (varyans homojenligi saglanmistir.)

#####################################
# GOREV 3 ADIM 2:
#*TODO: varsayim kontrolu sonrası uygun testi seciniz

#####################################

normallik ve varyans homojenligi varsayimlari saglandigi icin parametrik yöntem
tercih edilir (porportion_ztest uygulanacak) - varsayimlarin ikisi de saglandigi icin equal_var= True -

In [30]:
test_stat, p_value = proportions_ztest(reklama_tiklayan, reklami_goruntuleyen)
print("test degeri : %.4f , p value : %.4f " % (test_stat,p_value))

test degeri : 129.3305 , p value : 0.0000 


#####################################
# GOREV 3 ADIM 3:
* TODO: sonuclari yorumla

#####################################


p value < 0.05 yani en basta kurdugumuz H0 hipotezi reddedilir. 
control ve test olarak olusturulmus iki farkli tasarıma ait veri setlerinin ortalama purchase degeri 
hesaplandiginda meydana gelen farklilik tesadufi degildir, istatistiksel olarak anlamli bir farklilikdir.




#####################################
# GOREV 4: HIPOTEZ TESTININ GERCEKLESTIRILMESI
#####################################
# GOREV 4 ADIM 1:
* TODO: hangi test neden kullanildi, sebepler ?

#####################################


incelemelerimiz oran üzerinden oldugu için anahipotezimizi sorgulamak adına proprotion_ztest kullandık
normallik ve varyans homojenligi kısmında ise sirasiyla shapiro ve levene  testleri ile alt hipotezlerimizin 
dogrulugunu sinadik.

#####################################
# GOREV 4 ADIM 2:
* TODO: sonuclara gore aksiyon fikirleriniz?

#####################################


ana hipotezimizin p value'su 0.05 den kücük oldugu icin iki tasarımın ortalama purchase uzerinde farklı geri dosnusum 
oranlarina sahip olmasının tamamen tesaduften ibaret oldugunu saavunan H0 hipotezimizi reddederiz.bu sayede uygulamamizin 
basinda olctugumuz geri donusum oranlarından daha yuksek olan control veri setine uygulanan tasarımın bariz bir sekilde 
daha iyi sonuclar aciga cikardigi gorulmektedir.uygulamanin bu andan itibaren control 
veri setindeki haliyle kalmasi mantikli karar olacaktir.